----------------------------

# 추가 라벨링 시이작

In [ ]:
import pandas as pd
import numpy as numpy

In [ ]:
#attraction: museum, park,view,beautiful,amazing
#eat:restaurant, bar
#transport: great,train, subway,metro, bus,station,metre,underground, transport, access, location, centre, position,tube
#service: lovely, friendly, staff, helpful,balcony, room, breakfast, tidy, polite, reception

In [ ]:
attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'balcony', 'room', 'breakfast', 'tidy', 'polite', 'reception']

roomsize_keywords = ['tidy, narrows, narrow, compact, small']
noisy_keywords = ['noise','noisy','soundproofng','soundproof','sound','sounds']
dirty_keywords = ['dirty', 'uncleaned','cleaned', 'toilet', 'bathroom','pillow','bug','smell']
expensive_keywords = ['expensive','price','pricy','high','prices','payment']

------------------------

# noun_adj_pairs_raw(neg) 파일 먼저 이용!

In [ ]:
import pandas as pd
from collections import defaultdict, Counter

df = pd.read_csv('/content/noun_adj_pairs_raw(neg).csv')

print("원본 데이터")
print(df)
print("---")

# 정원님이 만든 키워드 딕셔너리
keywords_dict = {
    'attraction': ['museum', 'park', 'view', 'beautiful', 'amazing'],
    'eat': ['restaurant', 'bar'],
    'transport': ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube'],
    'service': ['lovely', 'friendly', 'staff', 'helpful', 'balcony', 'room', 'breakfast', 'tidy', 'polite', 'reception'],
    'roomsize': ['tiny', 'narrows', 'narrow', 'compact', 'small'],
    'noisy': ['noise', 'noisy', 'soundproofng', 'soundproof', 'sound', 'sounds'],
    'dirty': ['dirty', 'uncleaned', 'cleaned', 'toilet', 'bathroom', 'pillow', 'bug', 'smell'],
    'expensive': ['expensive', 'price', 'pricy', 'high', 'prices', 'payment']
}

# 라벨링 함수를 만들어보자!
def assign_label(text):
    if pd.isna(text): # 혹시 텍스트가 비어있는 경우 (NaN)를 대비
        return 'Not Applicable'
    text = str(text).lower() # 대소문자 상관없이 찾으려면 소문자로 다 바꿔주기. 및 문자열!

    for label, keywords in keywords_dict.items():
        for keyword in keywords:
            if keyword in text:
                    return label # 키워드를 찾으면 해당 라벨을 반환하고 끝!
    return 'Other/General' # 키워드를 하나도 못 찾으면 'Other/General'키워드


# 형용사와 명사를 띄어쓰기로 연결해서 하나의 문장처럼 만들기
df['combined_text'] = df['adj'].astype(str) + ' ' + df['noun'].astype(str)

# 새로운 'label' 컬럼을 추가해서 함수를 적용
df['label'] = df['combined_text'].apply(assign_label)

print(df[['adj', 'noun', 'combined_text', 'label']])

원본 데이터
             adj     noun
0       possible     site
1          small     room
2            big   window
3           high  ceiling
4           mini   fridge
...          ...      ...
285664      free  parking
285665    public   garage
285666       hot     week
285667      more      air
285668       hot      air

[285669 rows x 2 columns]
---
             adj     noun  combined_text          label
0       possible     site  possible site  Other/General
1          small     room     small room        service
2            big   window     big window  Other/General
3           high  ceiling   high ceiling      expensive
4           mini   fridge    mini fridge  Other/General
...          ...      ...            ...            ...
285664      free  parking   free parking     attraction
285665    public   garage  public garage  Other/General
285666       hot     week       hot week  Other/General
285667      more      air       more air  Other/General
285668       hot      air        h

bert 파일 이용

In [ ]:
import re
import string

# 1. 수동으로 정의한 키워드 리스트
attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'balcony', 'room', 'breakfast', 'tidy', 'polite', 'reception']
roomsize_keywords = ['tiny', 'narrows', 'narrow', 'compact', 'small']
noisy_keywords = ['noise','noisy','soundproofng','soundproof','sound','sounds']
dirty_keywords = ['dirty', 'uncleaned','cleaned', 'toilet', 'bathroom','pillow','bug','smell']
expensive_keywords = ['expensive','price','pricy','high','prices','payment']

# 모든 수동 키워드를 하나의 세트로 통합
manual_keywords = set(attraction_keywords + eat_keywords + transport_keywords + service_keywords + roomsize_keywords + noisy_keywords + dirty_keywords + expensive_keywords)

# 2. BERT 부정 리뷰 분석 파일 로드
df_bert_neg  = pd.read_csv("/content/negative_reviews_analysis_1.csv")

# 3. BERT 파일에서 모든 단어 추출
all_words = []
for pair_string in df_bert_neg['word_pair']:
    try:
        pair = eval(pair_string)
        if isinstance(pair, tuple) and len(pair) == 2:
            all_words.extend(pair)
    except (SyntaxError, NameError):
        continue

# 4. 수동 키워드 리스트에 없는 단어(새로운 키워드 후보) 찾기
new_keywords_candidates = [word for word in all_words if word not in manual_keywords]

# 5. 새로운 키워드 후보의 빈도수 계산 및 상위 20개 출력
new_keyword_counts = Counter(new_keywords_candidates)

print("\n--- BERT 분석에서 발견된 새로운 키워드 후보 (상위 20개) ---")
for word, count in new_keyword_counts.most_common(20):
    print(f"  - {word}: {count:,}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/negative_reviews_analysis_1.csv'

In [ ]:
#word_pair컬럼 사용해서, 빈도수 찾아내기!

In [ ]:
import pandas as pd
from collections import Counter
import re
import string

# 분석 대상 형용사 정의
# 명사에 대한 꾸밈에 집중할 만한 형용사들
# 'bed', 'hotel', 'service' 등은 명사에 더 가까워서 이 리스트에서 제외
target_adjectives = ['very', 'too', 'poor', 'good', 'bad', 'hot', 'old', 'cold', 'slow', 'other', 'more', 'much']

# 'word_pair' 컬럼에서 형용사와 명사 분리
def parse_word_pair(pair_string):
    try:
        if pair_string.startswith('(') and pair_string.endswith(')'): #'string' 리터럴이 아닌 실제 파이썬 튜플 형태인지 확인
            pair = eval(pair_string)
            if isinstance(pair, tuple) and len(pair) == 2:
                adj = str(pair[0]).strip() # 튜플 요소의 타입도 확인하여 불필요한 공백 제거
                noun = str(pair[1]).strip()
                return (adj, noun)
        return None
    except (SyntaxError, NameError, TypeError): # TypeError도 추가해서 좀 더 견고하게!
        return None

df_bert_neg['word_pair_tuple'] = df_bert_neg['word_pair'].apply(parse_word_pair)
df_bert_neg.dropna(subset=['word_pair_tuple'], inplace=True) # 유효하지 않은 쌍 제거

# 해당 타겟 형용사가 포함된 쌍만 필터링
for target_adj in target_adjectives:
    filtered_pairs = df_bert_neg[df_bert_neg['word_pair_tuple'].apply(lambda x: x[0] == target_adj)] # word_pair_tuple은 (adj, noun) 형태의 튜플이므로, adj가 target_adj와 같은지 확인

    # 해당 형용사가 꾸며주는 명사들만 추출
    nouns_for_adj = [pair[1] for pair in filtered_pairs['word_pair_tuple']]

    if nouns_for_adj: # 명사가 하나라도 있으면 출력
        noun_counts_for_adj = Counter(nouns_for_adj).most_common(10)
        print(f"'{target_adj}'이(가) 꾸며주는 명사 (총 {len(nouns_for_adj):,}개) :")
        for noun, count in noun_counts_for_adj:
            print(f"  - {noun}: {count:,}")
        print("--------------------------------------------------")


'very'이(가) 꾸며주는 명사 (총 10,698개) :
  - small: 1,298
  - poor: 718
  - noisy: 571
  - expensive: 537
  - slow: 348
  - bad: 332
  - good: 272
  - hot: 266
  - uncomfortable: 231
  - old: 224
--------------------------------------------------
'too'이(가) 꾸며주는 명사 (총 3,664개) :
  - small: 819
  - expensive: 386
  - hot: 302
  - much: 224
  - soft: 138
  - high: 131
  - warm: 96
  - noisy: 96
  - many: 91
  - hard: 88
--------------------------------------------------
'poor'이(가) 꾸며주는 명사 (총 1,678개) :
  - service: 150
  - breakfast: 114
  - quality: 93
  - value: 69
  - room: 60
  - wifi: 54
  - staff: 47
  - selection: 35
  - lighting: 35
  - connection: 32
--------------------------------------------------
'good'이(가) 꾸며주는 명사 (총 1,160개) :
  - value: 96
  - hotel: 44
  - sleep: 43
  - service: 36
  - room: 36
  - thing: 29
  - location: 27
  - breakfast: 26
  - experience: 26
  - idea: 23
--------------------------------------------------
'bad'이(가) 꾸며주는 명사 (총 1,207개) :
  - experience: 141
  - smel

```
갯수가 좀 적더라도 명사에 집중!


attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'balcony', 'room', 'breakfast', 'tidy', 'polite', 'reception', 'poor','slow','other','old','cold']
roomsize_keywords = ['tiny', 'narrows', 'narrow', 'compact', 'small']
noisy_keywords = ['noise','noisy','soundproofng','soundproof','sound','sounds']
dirty_keywords = ['dirty', 'uncleaned','cleaned', 'toilet', 'bathroom','pillow','bug','smell','bad']
expensive_keywords = ['expensive','price','pricy','high','prices','payment','more']



- very 형용사를 꾸며주는 역할이 너무 강해서 제외
- too 형용사를 꾸며주는 역할이 너무 강해서 제외
- poor -> service/breakfast/quality -> service
- good -> value/hotel/sleep -> ?
- bad -> experience/smell/room -> dirty
- hot -> water/food/breakfast -> ?
- old -> room/hotel/furniture -> service
- cold -> water/room/air -> service
- slow -> service/wifi/check -> service
- other -> hotel/room/guest -> service
- more -> expensive/comfortable/space -> expensive
- much 형용사를 꾸며주는 역할이 너무 강해서 제외

```
추가적인 의견,,,!

# "시설/어메니티" 관련 명사들을 위한 카테고리 강화
facility_amenity_keywords = ['hotel', 'room', 'bed', 'water', 'furniture', 'building', 'carpet', 'window', 'mattress', 'wifi', 'connection', 'elevator', 'lift', 'fridge', 'quality', 'value', 'lighting', 'tub', 'shower']

# "경험/상황" 관련 명사 카테고리 (추상적인 것 포함)
experience_condition_keywords = ['experience', 'sleep', 'thing', 'day', 'smell', 'response', 'time', 'space']

# "식사/음식" 관련 명사 카테고리
food_keywords = ['breakfast', 'food', 'drink', 'egg', 'selection']

이런식으로 더 나누는거보다는 이대로 유지하는게 낫게쬬,,?!??!


# 피드백 반영

```
facility_amenity_keywords = [
    'hotel', 'room', 'bed', 'water', 'furniture', 'building', 'carpet', 'window', 'mattress',
    'wifi', 'connection', 'elevator', 'lift', 'fridge', 'lighting', 'tub', 'shower',
    'air' # 'hot', 'cold'가 꾸미던 'air' 추가
]
roomsize_keywords = ['tiny', 'narrows', 'narrow', 'compact', 'small'] # 'room'은 service_keywords로 이동
noisy_keywords = ['noise','noisy','soundproofng','soundproof','sound','sounds','sleep'] # 'sleep'추가
dirty_keywords = ['dirty', 'uncleaned','cleaned', 'toilet', 'bathroom','pillow','bug','smell', 'bad'] # 'bad'를 청결도와 연관
service_keywords = [
    'lovely', 'friendly', 'staff', 'helpful', 'polite', 'reception', 'service', # 기본적인 서비스 요소
    'balcony', 'room', 'breakfast', 'tidy', # 'room', 'balcony'는 시설적 서비스, 'breakfast', 'tidy'는 서비스 품질 관련
    'poor', 'slow', 'old', 'cold', 'more', # BERT에서 나온 형용사/부사들을 서비스 및 환경 상태 평가 요소로 포함
    'value', 'quality', 'check', 'experience', 'response', # poor, good이 꾸미던 추상적인 명사 중 서비스와 연관
    'food', 'drink', 'egg', 'selection' # 식사/음식 관련 명사들 (호텔 서비스로 간주)
]
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
expensive_keywords = ['expensive','price','pricy','high','prices','payment'] # 'more'는 service_keywords로 이동

# 경험/상황 관련 키워드 : 조장님 피드백 반영
# 'thing', 'day', 'side', 'people', 'option', 'place', 'hand', 'minute', 'hour' 등은  'other'로 빠지도록 놔둠





# Q: other 처리 추가버전

- negative_reviews_analysis_1 파일 이용
- other 카테고리에서 상위권에 있는, 또는 의미가 명확한 단어들을 뽑아서 기존 카테고리에 추가

In [ ]:

df_pairs = pd.read_csv("/content/noun_adj_pairs_raw(neg).csv")


# 카테고리 라벨링 함수 정의
def assign_category_by_noun_priority(adj, noun):
    categories = []
    adj_lower = str(adj).lower() if pd.notna(adj) else ''
    noun_lower = str(noun).lower() if pd.notna(noun) else ''

    # 1순위: 명사(noun)가 어떤 키워드에 속하는지 확인
    for category, keywords in processed_keyword_categories.items():
        if any(kw in noun_lower for kw in keywords):
            if category not in categories:
                categories.append(category)

    # 2순위: 형용사(adj)가 명확한 카테고리 키워드일 때 (and 아직 추가되지 않은 카테고리에 한해)
    for category, keywords in processed_keyword_categories.items():
        if any(kw in adj_lower for kw in keywords):
            if category not in categories:
                categories.append(category)

    # 만약 아무 카테고리도 할당되지 않았다면 'other'
    return categories if categories else ['other']


# 5. 데이터프레임에 카테고리 컬럼 추가
df_pairs['categories'] = df_pairs.apply(lambda row: assign_category_by_noun_priority(row['adj'], row['noun']), axis=1)

# 6. 빈도수 계산
pair_counts = Counter(tuple(row) for row in df_pairs[['adj', 'noun']].values)

# 7. 'other' 카테고리 상위 쌍 확인
other_pairs_df = df_pairs[df_pairs['categories'].apply(lambda x: 'other' in x)].copy()

if not other_pairs_df.empty:
    other_pairs_df['count'] = other_pairs_df.apply(lambda row: pair_counts.get((row['adj'], row['noun']), 0), axis=1)
    print("\n### (참고) 라벨링 되지 않은 'other' 카테고리 상위 쌍 (빈도수 기준) ###")
    print(other_pairs_df.sort_values(by='count', ascending=False).head(10).to_string())
    print("----------------------------------------------------------------")

    distinct_other_pairs = other_pairs_df[['adj', 'noun']].drop_duplicates()
    print("\n### 라벨링 되지 않은 'other' 카테고리의 고유한 쌍들 (Distinct Pairs) ###")
    if len(distinct_other_pairs) > 10:
        print(f"총 {len(distinct_other_pairs)}개의 고유한 쌍 중 상위 10개:")
        print(distinct_other_pairs.head(10).to_string(index=False))
    else:
        print(f"총 {len(distinct_other_pairs)}개의 고유한 쌍:")
        print(distinct_other_pairs.to_string(index=False))

else:
    print("\n❗ 'other' 카테고리에 속하는 단어 쌍이 없습니다.")

print("\n---")
print("🎉 최종 라벨링 결과 (카테고리 컬럼 추가된 버전) 🎉")
print(df_pairs.head(10).to_string())

print("\n📈 각 카테고리별 쌍의 개수 📈")
all_assigned_categories = [cat for sublist in df_pairs['categories'] for cat in sublist]
print(Counter(all_assigned_categories))


### (참고) 라벨링 되지 않은 'other' 카테고리 상위 쌍 (빈도수 기준) ###
          adj  noun categories  count
100491  front  desk    [other]   1451
100348  front  desk    [other]   1451
207336  front  desk    [other]   1451
168335  front  desk    [other]   1451
207008  front  desk    [other]   1451
6156    front  desk    [other]   1451
6165    front  desk    [other]   1451
100124  front  desk    [other]   1451
100160  front  desk    [other]   1451
70      front  desk    [other]   1451
----------------------------------------------------------------

### 라벨링 되지 않은 'other' 카테고리의 고유한 쌍들 (Distinct Pairs) ###
총 47563개의 고유한 쌍 중 상위 10개:
      adj      noun
 possible      site
      bio    weapon
     next       day
     good       way
wonderful     waist
 peaceful    garden
 internal    garden
    audio recording
   single     level
    basic    coffee

---
🎉 최종 라벨링 결과 (카테고리 컬럼 추가된 버전) 🎉
        adj     noun                             categories
0  possible     site                                [other]
1     s

----------------------

# 최종 버전

```
[최종 버전] : 피드백 반영 + other 세부 단어 추가

facility_amenity_keywords = [
    'hotel', 'room', 'bed', 'water', 'furniture', 'building', 'carpet', 'window', 'mattress',
    'wifi', 'connection', 'elevator', 'lift', 'fridge', 'lighting', 'tub', 'shower',
    'air',
    'level','audio','recording'
]
roomsize_keywords = ['tiny', 'narrows', 'narrow', 'compact', 'small']
noisy_keywords = ['noise','noisy','soundproofng','soundproof','sound','sounds','sleep']
dirty_keywords = ['dirty', 'uncleaned','cleaned', 'toilet', 'bathroom','pillow','bug','smell', 'bad']
service_keywords = [
    'lovely', 'friendly', 'staff', 'helpful', 'polite', 'reception', 'service',
    'balcony', 'room', 'breakfast', 'tidy',
    'poor', 'slow', 'old', 'cold', 'more', #
    'value', 'quality', 'check', 'experience', 'response',
    'food', 'drink', 'egg', 'selection',
    'desk', 'day','coffee'
]
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
attraction_keywords = ['museum', 'park', 'view' 'beautiful', 'amazing',
'garden'
]
expensive_keywords = ['expensive','price','pricy','high','prices','payment']

경험/상황 관련 키워드 : 조장님 피드백 반영
'thing', 'way', 'side', 'people', 'option', 'place', 'hand', 'minute', 'hour' 등은  'other'로 빠지도록 놔둠